In [15]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import random
import scipy as sp
import numpy as np
import scipy.io
import cmath
from math import log
from scipy import fftpack
from numpy import sum,isrealobj,sqrt
from numpy.random import standard_normal
from numpy import savetxt
from google.colab import drive 
drive.mount('/content/gdrive')
################################################################################
def radioFadingChannel(nSample): 
  v_mph = 60 # velocity of either TX or RX, in miles per hour
  center_freq = 200e6 # RF carrier frequency in Hz
  Fs = nSample # sample rate of simulation
  N = 100 # number of sinusoids to sum
  v = v_mph * 0.44704 # convert to m/s
  fd = v*center_freq/3e8 # max Doppler shift
  t = np.arange(0, 1, 1/Fs) # time vector. (start, stop, step)*
  x = np.zeros(len(t))
  y = np.zeros(len(t))
  for i in range(N):
    alpha = (np.random.rand() - 0.5) * 2 * np.pi
    phi = (np.random.rand() - 0.5) * 2 * np.pi
    x = x + np.random.randn() * np.cos(2 * np.pi * fd * t * np.cos(alpha) + phi)
    y = y + np.random.randn() * np.sin(2 * np.pi * fd * t * np.cos(alpha) + phi)
  # z is the complex coefficient representing channel, you can think of this as a phase shift and magnitude scale
  z = (1/np.sqrt(N)) * (x + 1j*y) 
  z_mag = np.abs(z)
  return z
####################### Functions ############################################## 
def quadMean(x):
    y = np.array([abs(n)**2 for n in x])
    return y
###
def setMeanPower(signal, Power):
  y=0
  pZero = quadMean(signal)
  alpha = np.sqrt(Power/pZero)
  y = alpha*signal
  return y
###
def setSNR(signal, h, SNRdB):
  r = len(signal)
  y = np.zeros(r)
  snrSlin = 10**(SNRdB/10)
  pSignal = quadMean(h*signal)
  pNoise = pSignal/snrSlin     
  n = standard_normal(len(signal))+1j*standard_normal(len(signal))
  noise = standard_normal(len(signal))+1j*standard_normal(len(signal))
  n = setMeanPower(n, pNoise)
  noise = setMeanPower(noise, pNoise)
  y = h*signal + n
  return y, noise
################################################################################
L = 5120000
N = 64
M = 4
S = 4
SNR = np.arange(-20,1, 1)
signal = np.array([standard_normal(L)+1j*standard_normal(L) for i in range(len(SNR))])
H = np.array([radioFadingChannel(len(signal[1])) for i in range(len(SNR))])
Hot = np.array([setSNR(signal[i], H[i], SNR[i]) for i in range(len(SNR))])
H1t, H0t = Hot[:, 0,:], Hot[:, 1,:]
H1t, H0t = H1t.reshape(len(SNR), int(L/N), N), H0t.reshape(len(SNR), int(L/N), N)
np.random.seed(32)
for i in range(H1t.shape[0]):
  np.random.shuffle(H1t[i])
  np.random.shuffle(H0t[i])
H1, H0 = H1t.flatten(), H0t.flatten()
H1, H0 = H1.reshape(int(L*len(SNR)/(M*N*S)), N*M*S), H0.reshape(int(L*len(SNR)/(M*N*S)), N*M*S)
np.random.seed(42)
np.random.shuffle(H1)
np.random.shuffle(H0)
H1, H0 = np.abs(H1), np.abs(H0)
i = np.arange(S+1)*N*M
H1u, H0u = H1[:,i[0]:i[1]], H0[:,i[0]:i[1]]
H1x, H0x = H1[:,i[1]:i[2]], H0[:,i[1]:i[2]]
H1y, H0y = H1[:,i[2]:i[3]], H0[:,i[2]:i[3]]
H1z, H0z = H1[:,i[3]:i[4]], H0[:,i[3]:i[4]]
################################################################################
def matrix_generation(X1, X0, length, n_antennas, n_exemples): 
  p = int(n_exemples/2)
  X1=X1[0:p,:]
  X0=X0[0:p,:]
  y1 = np.ones((p,1))
  y0 = np.zeros((p,1))
  X, y = np.vstack((X1,X0)), np.vstack((y1,y0))
  X = X.reshape(X.shape[0], int(length*n_antennas), 1).astype('float32')
  # Data splitting  
  Xc_train, Xc_test, yc_train, yc_test = train_test_split(X, y, random_state=42, train_size=.8) 
  # Encoding Categorical Data
  Yc_train = np_utils.to_categorical(yc_train, 2)
  Yc_test = np_utils.to_categorical(yc_test, 2)
  return Xc_train, Xc_test, Yc_train, Yc_test
################################################################################
n_exemples = 140000
###
Xcux_train, Xcux_test, Ycux_train, Ycux_test = matrix_generation(H1u, H0u, N, M, n_exemples)
Xcxx_train, Xcxx_test, Ycxx_train, Ycxx_test = matrix_generation(H1x, H0x, N, M, n_exemples)
Xcyx_train, Xcyx_test, Ycyx_train, Ycyx_test = matrix_generation(H1y, H0y, N, M, n_exemples)
Xczx_train, Xczx_test, Yczx_train, Yczx_test = matrix_generation(H1z, H0z, N, M, n_exemples)
Ycx_train, Ycx_test = Ycxx_train, Ycxx_test
################################################################################
np.save('/content/gdrive/My Drive/Xcu_train64x.npy', Xcux_train)
np.save('/content/gdrive/My Drive/Xcx_train64x.npy', Xcxx_train)
np.save('/content/gdrive/My Drive/Xcy_train64x.npy', Xcyx_train)
np.save('/content/gdrive/My Drive/Xcz_train64x.npy', Xczx_train)
np.save('/content/gdrive/My Drive/Xcu_test64x.npy', Xcux_test)
np.save('/content/gdrive/My Drive/Xcx_test64x.npy', Xcxx_test)
np.save('/content/gdrive/My Drive/Xcy_test64x.npy', Xcyx_test)
np.save('/content/gdrive/My Drive/Xcz_test64x.npy', Xczx_test)
np.save('/content/gdrive/My Drive/Yc_train64x.npy', Ycx_train)
np.save('/content/gdrive/My Drive/Yc_val64x.npy', Ycx_test)

array([   0,  256,  512,  768, 1024])